第一步：整理数据集
首先一个问题就是如何生成验证集，文中的数据集有验证集，而我们下载的是没用的，所以需要自己去生成
不过还是先从读取图片和csv文件开始吧

In [ ]:
from PIL import Image, ImageOps
import os
import numpy as np
import pandas as pd
import shutil
import collections
import math
import itertools
import random
import csv
import torch
from torch import Tensor
from d2l import torch as d2l
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import WeightedRandomSampler
from mymodel import MMANet



In [ ]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [ ]:
# 一些变量
#文件路径
data_dir = './dataset/'
batch_size = 4
valid_ratio = 0.1

在读取文件时，还有一个任务就是修改图片尺寸，使其成为512x512的格式

In [ ]:
def read_iamge(data_dir, fname, image_size=512):
    img = Image.open(os.path.join(data_dir, fname))
    # 开始修改尺寸
    w, h = img.size
    long = max(w, h)
    # 按比例缩放成512
    w, h = int(w/long*image_size), int(h/long*image_size)
    # 压缩并插值
    img = img.resize((w, h), Image.ANTIALIAS)
    # 然后是给短边扩充，使用ImageOps.expand
    delta_w, delta_h = image_size - w, image_size - h
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
    # 转化成np.array再返回
    return np.array(ImageOps.expand(img, padding).convert("RGB"))

training_dir = './dataset/boneage-training-dataset/'
this_pic = '1377.png'
convert_pic = read_iamge(training_dir, this_pic)
convert_pic.shape

将图片修改成了(512, 512, 3)

csv文件的读取采用pandas.readcsv,读出的是DataFrame格式，用df标记,将id和age提取出来当做dict处理

In [ ]:
training_csv = pd.read_csv(os.path.join(data_dir, 'boneage-training-dataset.csv'))
# training_dict = dict(training_csv['BoneAge'])
# training_dict = {}
# for index, item in list(training_csv['ID']), list(training_csv['BoneAge']):
    # training_dict[index] = item
# training_dict.values()
# collections.Counter(training_dict.values()).most_common()[-1][1]
# list(training_csv['ID'])
# training_dict = zip(list(training_csv['ID']), list(training_csv['BoneAge']))
# training_dict = dict(zip(list(training_csv['ID']), list(training_csv['BoneAge'])))
len(training_csv)

如何将验证集从原始的训练集中拆分出来。首先定义一个复制文件的函数

In [ ]:
def copyfile(fname, target_dir):
    """将文件复制到指定文件夹"""
    os.makedirs(target_dir, exist_ok=True)
    shutil.copy(fname, target_dir)

In [ ]:
# 生成一个小数据集
def reorg_train(data_dir, DF, ratio):
    # 取多少的样本
    n = int(len(DF)*ratio)
    # 先写上cvs文件的列名
    valid_list = [['ID', 'BoneAge', 'Male']]
    # 在dataframe中随机遍历n个
    for index, row in DF.sample(int(n)).iterrows():
        # 获取当前文件路径
        filename = os.path.join(data_dir, 'boneage-training-dataset', f"{row['ID']}.png")
        # 复制到指定文件夹
        copyfile(filename, os.path.join(data_dir, 'small-training-dataset'))
        # 将该行数据添加至列表
        valid_list.append([row['ID'], row['BoneAge'], row['Male']])
    # 将list的内容写入csv文件
    with open('./dataset/small-training-dataset.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for row in valid_list:
            writer.writerow(row)
    # # 读取验证集的csv，开始生成数据集，但是数据集必然很大
    # for train_file in os.listdir(os.path.join(data_dir, 'boneage-training-dataset')):
    #     ID = train_file.split('.')[0]
    #     fname = os.path.join(data_dir, 'boneage-training-dataset', train_file)
    return 0
reorg_train(data_dir, training_csv, 0.01)

定义reorg_train_valid函数来将验证集从原始的训练集中拆分出来

In [ ]:
def reorg_train_valid(data_dir, DF, valid_ratio):
    """将验证集从原始的数据集中拆分出来"""
    # 取多少的样本
    n = int(len(DF)*valid_ratio)
    # 先写上cvs文件的列名
    valid_list = [['ID', 'BoneAge', 'Male']]
    # 在dataframe中随机遍历n个
    for index, row in DF.sample(int(n)).iterrows():
        # 获取当前文件路径
        filename = os.path.join(data_dir, 'boneage-training-dataset', f"{row['ID']}.png")
        # 复制到指定文件夹
        copyfile(filename, os.path.join(data_dir, 'boneage-valid-dataset'))
        # 将该行数据添加至列表
        valid_list.append([row['ID'], row['BoneAge'], row['Male']])
    # 将list的内容写入csv文件
    with open('./dataset/boneage-valid-dataset.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for row in valid_list:
            writer.writerow(row)
    # # 读取验证集的csv，开始生成数据集，但是数据集必然很大
    # for train_file in os.listdir(os.path.join(data_dir, 'boneage-training-dataset')):
    #     ID = train_file.split('.')[0]
    #     fname = os.path.join(data_dir, 'boneage-training-dataset', train_file)
    return 0

# reorg_train_valid(data_dir, training_csv, 0.01)

预处理数据集

In [ ]:
valid_csv = pd.read_csv(os.path.join(data_dir, 'boneage-valid-dataset.csv'))
test_csv = pd.read_csv(os.path.join(data_dir, 'boneage-test-dataset.csv'))
small_csv = pd.read_csv(os.path.join(data_dir, 'small-training-dataset.csv'))
# 数据预处理,数字标准化，非数字型转化
boneage_mean = training_csv['BoneAge'].mean()
boneage_std = training_csv['BoneAge'].std()
training_csv['zscore'] = training_csv['BoneAge'].map(lambda x: (x - boneage_mean) / boneage_std)
training_csv['Male']  = training_csv['Male'].astype('float32')
training_csv['BoneAge'] = training_csv['BoneAge'].astype('float32')

small_mean = small_csv['BoneAge'].mean()
small_std = small_csv['BoneAge'].std()
small_csv['zscore'] = small_csv['BoneAge'].map(lambda x: (x - small_mean) / small_std)
small_csv['Male']  = small_csv['Male'].astype('float32')
small_csv['BoneAge'] = small_csv['BoneAge'].astype('float32')

valid_csv['Male'] = valid_csv['Male'].astype('float32')
valid_csv['BoneAge'] = valid_csv['BoneAge'].astype('float32')
valid_csv

第二步：图像增广
使用图像增广来解决过拟合的问题

In [ ]:
# 导入用于图像增广的包
from albumentations.augmentations.transforms import Lambda, RandomBrightnessContrast
from albumentations.augmentations.geometric.transforms import ShiftScaleRotate, HorizontalFlip
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.augmentations.crops.transforms import RandomResizedCrop
from albumentations import Compose

import cv2
from torchvision import transforms

In [ ]:
# 随机删除一个图片上的像素，p为执行概率，scale擦除部分占据图片比例的范围，ratio擦除部分的长宽比范围
randomErasing = transforms.RandomErasing(scale=(0.02, 0.08), ratio=(0.5, 2), p=0.8)

def randomErase(image, **kwargs):
    return randomErasing(image)


x = torch.rand((512, 512, 3))

randomErase(x)

In [ ]:
# 标准化每个通道
def sample_normalize(image, **kwargs):
    image = image//255
    channel = image.shape[2]
    mean, std = image.reshape((-1, channel)).mean(axis = 0), image.reshape((-1, channel)).std(axis = 0)
    return (image - mean)/(std + 1e-3)

sample_normalize(x)

In [ ]:

# 训练集的图像增广

transform_train = Compose([
    # 随机大小裁剪，512为调整后的图片大小，（0.5,1.0）为scale剪切的占比范围，概率p为0.5
    RandomResizedCrop(512, 512, (0.5, 1.0), p = 0.5),
    # ShiftScaleRotate操作：仿射变换，shift为平移，scale为缩放比率，rotate为旋转角度范围，border_mode用于外推法的标记，value即为padding_value，前者用到的，p为概率
    ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20, border_mode=cv2.BORDER_CONSTANT, value=0.0, p=0.8),
    # 水平翻转
    HorizontalFlip(p=0.5),
    # 概率调整图片的对比度
    RandomBrightnessContrast(p=0.8, contrast_limit=(-0.3, 0.2)),
    # 标准化
    Lambda(image=sample_normalize), 
    # 将图片转化为tensor类型
    ToTensorV2(),
    # 做随机擦除
    Lambda(image=randomErase)
])

transform_train(image = convert_pic)['image']

遗留问题：验证集是从训练集里面提取出来的，然而并没有从训练集中抽离出来，所以是失败的，下面写一个k折交叉验证中获得验证集的方法

In [ ]:
def get_k_fole_data(k, i, X, y):
    # 确保k合法
    assert k>1
    fold_size = X.shape[0] // i
    X_train, y_train = 0, 0
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx, :]
        if i == j :
            X_valid, y_valid = X_part, y_part
        elif X_train == None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train ,X_valid, y_valid

训练

In [ ]:
# 罚函数
def L1_penalty(net, alpha):
    loss = 0
    for param in net.MLP.parameters():
        loss += torch.sum(torch.abs(param))

    return alpha*loss

读取数据集

In [ ]:
# 重写getitem函数，这样可以将这个类类似于DF
class BAATrainDatasets(Dataset):
    def __init__(self, df, file_path) -> None:
        super().__init__()
        self.df = df
        self.file_path = file_path
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        num = int(row['ID'])
        return (transform_train(image = read_iamge(self.file_path, f"{num}.png"))['image'], Tensor([row['Male']])), row['zscore']

    def __len__(self):
        return len(self.df)

train_dataset = BAATrainDatasets(training_csv, os.path.join(data_dir, 'boneage-training-dataset'))
small_dataset = BAATrainDatasets(small_csv, os.path.join(data_dir, 'small-training-dataset'))
small_dataset.__getitem__(1)

In [ ]:
# 定义损失函数
# loss = nn.CrossEntropyLoss(reduction="none")
loss = nn.L1Loss(reduction='sum')

In [ ]:
# 验证集的数据处理
transform_valid = Compose([
    Lambda(image=sample_normalize),
    ToTensorV2()
])

In [ ]:
# 读取验证集
class BAAValidDatasets(Dataset):

    def __init__(self, df, file_path) -> None:
        super().__init__()
        self.df = df
        self.file_path = file_path

    def __getitem__(self, index):
        row = self.df.iloc[index]
        num = int(row['ID'])
        return (transform_valid(image=read_iamge(self.file_path, f"{num}.png"))['image'], Tensor([row['Male']])), row['BoneAge']

    def __len__(self):
        return len(self.df)

In [ ]:
# 读入数据集
train_dataset = BAATrainDatasets(training_csv, os.path.join(data_dir, 'boneage-training-dataset'))
valid_dataset = BAAValidDatasets(valid_csv, os.path.join(data_dir, 'boneage-valid-dataset'))


In [ ]:
# 配置采样器
train_sampler = torch.utils.data.distributed.DistributedSampler(
    train_dataset,
    num_replicas=1,   # 检索参与复制的设备数
    rank=0,  # 检索当前进程的复制序号，序数范围从 0 到 xrt_world_size()-1
    shuffle=True)
valid_sampler = torch.utils.data.distributed.DistributedSampler(
    valid_dataset,
    num_replicas=1,
    rank=0,
    shuffle=True
)
small_sampler = torch.utils.data.distributed.DistributedSampler(
    small_dataset,
    num_replicas=1,   # 检索参与复制的设备数
    rank=0,  # 检索当前进程的复制序号，序数范围从 0 到 xrt_world_size()-1
    shuffle=True)

In [ ]:
# 读取数据
train_iter = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=4,
    sampler=train_sampler,
    drop_last=True
)
valid_iter = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=4,
    sampler=valid_sampler,
    drop_last=True
)
small_iter = torch.utils.data.DataLoader(
    small_dataset,
    batch_size=4,
    sampler=small_sampler,
    drop_last=True
)

In [ ]:
# 获取网络
def get_net():
    net = Ensemble()
    return net


In [ ]:
def accuracy(y_hat, y):
    """Compute the number of correct predictions.

    Defined in :numref:`sec_softmax_scratch`"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = d2l.argmax(y_hat, axis=1)
    cmp = d2l.astype(y_hat, y.dtype) == y
    return float(d2l.reduce_sum(d2l.astype(cmp, y.dtype)))

In [ ]:
# 定义一个在多个GPU上小批量训练的函数

def train_batch(net, X, gender, y, loss, trainer):
    # 将数据布置在设备上
    # if isinstance(X, list):
    #     # BERT中所需
    #     # X = [x.to(devices[0]) for x in X]
    #     X = [x.cuda() for x in X]
    # else:
    gender = gender.cuda()
    X = X.cuda()
    y = torch.reshape(y, [4, 1])
    y = y.cuda()
    # gender = gender.cuda()
    # 开启微调
    net.train()
    # 将梯度清0
    trainer.zero_grad()
    X = X.float()
    pred = net(X, gender)
    pred = pred.squeeze()
    l = loss(pred, y)
    train_loss_sum = l + L1_penalty(net, 1e-5)
    train_loss_sum.backward()
    # l.sum().backward()
    trainer.step()
    # train_loss_sum = l.sum()
    # print(f'loss_sum：{train_loss_sum}')
    return train_loss_sum

In [ ]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
    """Compute the accuracy for a model on a dataset using a GPU.

    Defined in :numref:`sec_lenet`"""
    if isinstance(net, nn.Module):
        net.eval()  # Set the model to evaluation mode
        if not device:
            device = next(iter(net.parameters())).device
    # No. of correct predictions, no. of predictions
    metric = d2l.Accumulator(2)

    with torch.no_grad():
        for X, y in data_iter:
            valid_labels = y
            valid_feature, valid_gender = X[0], X[1]
            if isinstance(X, list):
                # Required for BERT Fine-tuning (to be covered later)
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(d2l.accuracy(net(valid_feature, valid_gender), valid_labels), d2l.size(valid_labels))
    return metric[0] / metric[1]

In [ ]:
# 定义训练函数

def train(net, train_iter, valid_iter, num_epoch, lr, wd, devices, lr_period, lr_decay):
    # 定义Adam优化器
    trainer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=wd)
    # 每隔几个周期，学习率就衰减
    scheduler = torch.optim.lr_scheduler.StepLR(trainer, lr_period, lr_decay)
    num_batches, timer = len(train_iter), d2l.Timer()
    legend = ['train loss']
    if valid_iter is not None:
        legend.append('valid acc')
    # 可视化
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epoch], legend=legend)
    # 在多个GPU上部署
    net = net.to(device=try_gpu())

    for epoch in range(num_epoch):
        net.train()
        # 可视化函数
        metric = d2l.Accumulator(3)
        for i, data in enumerate(train_iter):
            timer.start()
            labels = data[1]
            feature, gender = data[0]
            l = train_batch(net, feature, gender, labels, loss, trainer)
            metric.add(l, labels.shape[0])
            timer.stop()
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches -1:
                # 在animator中加入批次，以及平均损失和平均精确度
                animator.add(epoch + (i + 1) / num_batches,
                             (metric[0] / metric[1], None)
                             )
        # 若训练集不为空，则测试一下当前模型的精度，且将其在animator中显示
        if valid_iter is not None:
            for idx, data in enumerate(valid_iter):
                # valid_labels = data[1]
                # valid_feature, valid_gender = data[0]
                valid_acc = evaluate_accuracy_gpu(net, valid_iter)
                animator.add(epoch + 1 , (None, valid_acc))
            
        # 执行完一个小批次，使scheduler执行一下
        scheduler.step()
    measures = (f'train loss {metric[0] / metric[1]:.3f}')
    if valid_iter is not None:
        measures += f', valid acc {valid_acc:.3f}'
    print(measures + f'\n{measures[1] * num_epoch / timer.sum():.1f}'
          f' examples/sec on {str(devices)}')

In [ ]:
devices, num_epochs, lr, wd = try_gpu(), 10, 2e-4, 5e-4
lr_period, lr_decay, net = 10, 0.5, get_net()

In [ ]:
train(net, small_iter, valid_iter, num_epochs, lr, wd, devices, lr_period, lr_decay)